[View in Colaboratory](https://colab.research.google.com/github/TonyIssacJames/EIP_public/blob/master/DNST_CIFAR10_AUG.ipynb)

In [1]:
# https://keras.io/
#!pip install -q keras
import keras

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [3]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [4]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 20
compression = 0.6
dropout_rate = 0.3

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
# Dense Block
def add_denseblock(input, num_filter = 20, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [7]:
def add_transition(input, num_filter = 20, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [8]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [9]:
#num_filter = 16
#Sdropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Fourth_Block = add_denseblock(Third_Transition, num_filter, dropout_rate)
Fourth_Transition = add_transition(Fourth_Block, num_filter, dropout_rate)

#Fifth_Block = add_denseblock(Fourth_Transition, num_filter, dropout_rate)
#Fifth_Transition = add_transition(Fifth_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Fourth_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [10]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 20)   540         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 20)   80          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 20)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [11]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [12]:
#Callback for saving the best model
from keras.callbacks import ModelCheckpoint
filepath= "weights_goo_tr_06.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [13]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 227s 5ms/step - loss: 1.7558 - acc: 0.3227 - val_loss: 1.8294 - val_acc: 0.3834

Epoch 00001: val_acc improved from -inf to 0.38340, saving model to weights_goo_tr_06.best.hdf5
Epoch 2/10
50000/50000 [==============================] - 192s 4ms/step - loss: 1.3666 - acc: 0.4930 - val_loss: 1.5321 - val_acc: 0.4888

Epoch 00002: val_acc improved from 0.38340 to 0.48880, saving model to weights_goo_tr_06.best.hdf5
Epoch 3/10
50000/50000 [==============================] - 192s 4ms/step - loss: 1.1764 - acc: 0.5747 - val_loss: 1.8866 - val_acc: 0.4825

Epoch 00003: val_acc did not improve from 0.48880
Epoch 4/10
50000/50000 [==============================] - 192s 4ms/step - loss: 1.0626 - acc: 0.6185 - val_loss: 1.4223 - val_acc: 0.5652

Epoch 00004: val_acc improved from 0.48880 to 0.56520, saving model to weights_goo_tr_06.best.hdf5
Epoch 5/10
50000/50000 [==========================

In [14]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 192s 4ms/step - loss: 0.7533 - acc: 0.7356 - val_loss: 1.5391 - val_acc: 0.5887

Epoch 00001: val_acc did not improve from 0.68260
Epoch 2/10
50000/50000 [==============================] - 192s 4ms/step - loss: 0.7271 - acc: 0.7457 - val_loss: 1.1069 - val_acc: 0.6670

Epoch 00002: val_acc did not improve from 0.68260
Epoch 3/10
50000/50000 [==============================] - 191s 4ms/step - loss: 0.7067 - acc: 0.7536 - val_loss: 0.8444 - val_acc: 0.7385

Epoch 00003: val_acc improved from 0.68260 to 0.73850, saving model to weights_goo_tr_06.best.hdf5
Epoch 4/10
50000/50000 [==============================] - 191s 4ms/step - loss: 0.6791 - acc: 0.7638 - val_loss: 1.8312 - val_acc: 0.5982

Epoch 00004: val_acc did not improve from 0.73850
Epoch 5/10
50000/50000 [==============================] - 191s 4ms/step - loss: 0.6593 - acc: 0.7704 - val_loss: 0.9240 - val_acc: 0.7134

Epoch 

In [15]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 191s 4ms/step - loss: 0.5613 - acc: 0.8057 - val_loss: 0.9843 - val_acc: 0.7282

Epoch 00001: val_acc did not improve from 0.75110
Epoch 2/10
50000/50000 [==============================] - 191s 4ms/step - loss: 0.5485 - acc: 0.8113 - val_loss: 2.5150 - val_acc: 0.5521

Epoch 00002: val_acc did not improve from 0.75110
Epoch 3/10
50000/50000 [==============================] - 191s 4ms/step - loss: 0.5322 - acc: 0.8170 - val_loss: 0.8983 - val_acc: 0.7519

Epoch 00003: val_acc improved from 0.75110 to 0.75190, saving model to weights_goo_tr_06.best.hdf5
Epoch 4/10
50000/50000 [==============================] - 191s 4ms/step - loss: 0.5275 - acc: 0.8182 - val_loss: 1.2920 - val_acc: 0.6963

Epoch 00004: val_acc did not improve from 0.75190
Epoch 5/10
50000/50000 [==============================] - 191s 4ms/step - loss: 0.5172 - acc: 0.8210 - val_loss: 1.0056 - val_acc: 0.7308

Epoch 

In [16]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 192s 4ms/step - loss: 0.4618 - acc: 0.8418 - val_loss: 0.9120 - val_acc: 0.7623

Epoch 00001: val_acc did not improve from 0.76980
Epoch 2/10
50000/50000 [==============================] - 192s 4ms/step - loss: 0.4543 - acc: 0.8433 - val_loss: 1.7771 - val_acc: 0.6346

Epoch 00002: val_acc did not improve from 0.76980
Epoch 3/10
50000/50000 [==============================] - 191s 4ms/step - loss: 0.4487 - acc: 0.8473 - val_loss: 1.4239 - val_acc: 0.6792

Epoch 00003: val_acc did not improve from 0.76980
Epoch 4/10
50000/50000 [==============================] - 191s 4ms/step - loss: 0.4411 - acc: 0.8497 - val_loss: 0.8501 - val_acc: 0.7746

Epoch 00004: val_acc improved from 0.76980 to 0.77460, saving model to weights_goo_tr_06.best.hdf5
Epoch 5/10
50000/50000 [==============================] - 191s 4ms/step - loss: 0.4360 - acc: 0.8513 - val_loss: 1.0727 - val_acc: 0.7353

Epoch 

In [17]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 191s 4ms/step - loss: 0.3997 - acc: 0.8615 - val_loss: 1.1637 - val_acc: 0.7255

Epoch 00001: val_acc did not improve from 0.79430
Epoch 2/10
50000/50000 [==============================] - 191s 4ms/step - loss: 0.3903 - acc: 0.8663 - val_loss: 0.7309 - val_acc: 0.7955

Epoch 00002: val_acc improved from 0.79430 to 0.79550, saving model to weights_goo_tr_06.best.hdf5
Epoch 3/10
50000/50000 [==============================] - 191s 4ms/step - loss: 0.3890 - acc: 0.8659 - val_loss: 1.1045 - val_acc: 0.7438

Epoch 00003: val_acc did not improve from 0.79550
Epoch 4/10
50000/50000 [==============================] - 191s 4ms/step - loss: 0.3822 - acc: 0.8676 - val_loss: 1.5779 - val_acc: 0.6674

Epoch 00004: val_acc did not improve from 0.79550
Epoch 5/10
50000/50000 [==============================] - 191s 4ms/step - loss: 0.3796 - acc: 0.8703 - val_loss: 1.3790 - val_acc: 0.7131

Epoch 

In [18]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 15s 2ms/step
Test loss: 0.6737625888586044
Test accuracy: 0.8215


In [19]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model_original_tr_06.h5")
print("Saved model to disk")

Saved model to disk


In [21]:
from keras.models import load_model

new_model = load_model("weights_goo_tr_06.best.hdf5")
score = new_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 18s 2ms/step
Test loss: 0.6737625888586044
Test accuracy: 0.8215


In [0]:
#from google.colab import files

#files.download('DNST_model_original_tr_04.h5')